#### Connect to Google drive

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#### Check for the "stock" data

In [9]:
import os
os.listdir('/content/gdrive/Shared drives/marketdata/stock')


['2020-09-21.zip', '2020-09-21']

In [10]:
source_path = '/content/gdrive/Shared drives/marketdata/stock/'
dest_path = '/content/gdrive/Shared drives/marketdata/stock/'

#### Unzip the .zip file

In [11]:
os.chdir(source_path)

In [12]:
'''
from zipfile import ZipFile

for zipObj in os.listdir():
    if '.zip' in zipObj:
        with ZipFile(zipObj, 'r') as zipObj:
        # Extract all the contents of zip file in current directory
            zipObj.extractall(dest_path)
'''

#### Filter out each stock symbol

In [14]:
import pandas as pd
import numpy
import datetime as dt

In [125]:
def numtotime(num):
    ms = num % 1000
    num //= 1000
    s = num % 100
    num //= 100
    mi = num % 100
    num //= 100
    h = num % 100
    num = num // 100
    # print(h, mi, s, ms)
    return dt.datetime(2020, 9, 21, h, mi, s, ms) # TODO

#### Filter TSE

In [15]:
def filter_tse(date = "2020-09-21"):
    df_lob_dict = {}
    df_tick_dict = {}   
    df_lob = pd.read_csv(os.path.join(dest_path, date, "tse_lob.csv"), header = None)
    df_tick = pd.read_csv(os.path.join(dest_path, date, "tse_tick.csv"), header = None)
    col_lob = ['time', 'index_ori', 'stock', 'bid1_price', 'bid1_qty','bid2_price', 'bid2_qty',\
              'bid3_price', 'bid3_qty','bid4_price', 'bid4_qty','bid5_price', 'bid5_qty',\
              'ask1_qty', 'ask1_price','ask2_qty', 'ask2_price','ask3_qty', 'ask3_price',\
              'ask4_qty', 'ask4_price','ask5_qty', 'ask5_price', 'n1', 'n2']
    col_tick = ['time', 'index_ori', 'stock', 'time2', 'price', 'qty', 'n1', 'n2', 'n3', 'n4']      
    df_lob.columns = col_lob
    df_tick.columns = col_tick
    df_lob = df_lob.drop(['n1', 'n2', 'index_ori'], axis = 1)
    df_tick = df_tick.drop(['n1','n2','n3','n4','time2', 'index_ori'], axis = 1)
    df_lob = df_lob[df_lob['time'] >= 90000000]
    df_tick = df_tick[df_tick['time'] >= 90000000] 
    df_lob.stock = df_lob.stock.str.strip()
    df_tick.stock = df_tick.stock.str.strip()
    # TODO
    # stock_list = df_lob.stock.str.strip().unique()
    stock_list = ["6462", "6278", "6488", "3037"]
    # df_lob['time'] = df_lob['time'].apply(func = numtotime)
    # df_tick['time'] = df_tick['time'].apply(func = numtotime)
    # df_lob.drop
    for stock in stock_list:
        if (os.path.isdir(os.path.join(dest_path, date, stock))):
            pass
        else:
            os.mkdir(os.path.join(dest_path, date, stock))
        df_lob_dict[stock] = df_lob[df_lob.stock==stock]
        df_tick_dict[stock] = df_tick[df_tick.stock==stock]
        if len(df_lob_dict[stock]) >100:
            df_lob_dict[stock].to_csv(os.path.join(dest_path, date, stock, "tse_lob_" + stock +  ".csv"), index=False)
            df_tick_dict[stock].to_csv(os.path.join(dest_path, date, stock, "tse_tick_" + stock + ".csv"), index=False)
    return df_lob_dict, df_tick_dict

In [16]:
df_lob_dict, df_tick_dict = filter_tse()

In [17]:
df_lob_dict

{'3037':               time stock  bid1_price  ...  ask4_price  ask5_qty  ask5_price
 85090     90000651  3037        78.0  ...        78.4        17        78.5
 85107     90000791  3037        78.0  ...        78.4        17        78.5
 85118     90000853  3037        78.0  ...        78.4        17        78.5
 85119     90000856  3037        78.0  ...        78.4        17        78.5
 85126     90000867  3037        78.0  ...        78.4        17        78.5
 ...            ...   ...         ...  ...         ...       ...         ...
 4806822  132942325  3037        78.9  ...        79.3       197        79.4
 4807240  132947338  3037        78.9  ...        79.3       197        79.4
 4807681  132952350  3037        78.9  ...        79.3       197        79.4
 4808141  132957364  3037        78.9  ...        79.3       197        79.4
 4808477  133000000  3037        78.8  ...        79.2       162        79.3
 
 [35182 rows x 22 columns],
 '6278':               time stock  bid

In [18]:
df_tick_dict

{'3037':              time stock  price   qty
 73162    90000651  3037   78.0   256
 73174    90000903  3037   78.1     2
 73175    90000936  3037   78.1     2
 73196    90001177  3037   78.0     1
 73197    90001189  3037   78.0     4
 ...           ...   ...    ...   ...
 800124  132942325  3037   79.0  1781
 800533  132947338  3037   79.0  1782
 800965  132952350  3037   79.0  1791
 801418  132957364  3037   79.0  1862
 801748  133000000  3037   78.9  1868
 
 [8896 rows x 4 columns], '6278':              time stock  price  qty
 73770    90007181  6278  123.5   32
 78436    90034747  6278  123.5    2
 78518    90035287  6278  123.5    1
 78520    90035315  6278  123.5    2
 80694    90048173  6278  123.5    1
 ...           ...   ...    ...  ...
 800235  132943792  6278  119.0  462
 800671  132948805  6278  119.0  464
 801099  132953819  6278  119.0  471
 801589  132958834  6278  119.0  474
 801995  133000000  6278  119.0  474
 
 [1619 rows x 4 columns], '6462': Empty DataFrame
 Colu

#### Filter OTC

In [19]:
def filter_otc(date = "2020-09-21"):
    df_lob_dict = {}
    df_tick_dict = {}   
    df_lob = pd.read_csv(os.path.join(dest_path, date, "otc_lob.csv"), header = None)
    df_tick = pd.read_csv(os.path.join(dest_path, date, "otc_tick.csv"), header = None)
    col_lob = ['time', 'index_ori', 'stock', 'bid1_price', 'bid1_qty','bid2_price', 'bid2_qty',\
              'bid3_price', 'bid3_qty','bid4_price', 'bid4_qty','bid5_price', 'bid5_qty',\
              'ask1_qty', 'ask1_price','ask2_qty', 'ask2_price','ask3_qty', 'ask3_price',\
              'ask4_qty', 'ask4_price','ask5_qty', 'ask5_price', 'n1', 'n2']
    col_tick = ['time', 'index_ori', 'stock', 'time2', 'price', 'qty', 'n1', 'n2', 'n3', 'n4']      
    df_lob.columns = col_lob
    df_tick.columns = col_tick
    df_lob = df_lob.drop(['n1', 'n2', 'index_ori'], axis = 1)
    df_tick = df_tick.drop(['n1','n2','n3','n4','time2', 'index_ori'], axis = 1)
    df_lob = df_lob[df_lob['time'] >= 90000000]
    df_tick = df_tick[df_tick['time'] >= 90000000] 
    df_lob.stock = df_lob.stock.str.strip()
    df_tick.stock = df_tick.stock.str.strip()
    # TODO
    # stock_list = df_lob.stock.str.strip().unique()
    stock_list = ["6462", "6488"]
    # df_lob['time'] = df_lob['time'].apply(func = numtotime)
    # df_tick['time'] = df_tick['time'].apply(func = numtotime)
    # df_lob.drop
    for stock in stock_list:
        if (os.path.isdir(os.path.join(dest_path, date, stock))):
            pass
        else:
            os.mkdir(os.path.join(dest_path, date, stock))
        df_lob_dict[stock] = df_lob[df_lob.stock==stock]
        df_tick_dict[stock] = df_tick[df_tick.stock==stock]
        if len(df_lob_dict[stock]) > 100:
            df_lob_dict[stock].to_csv(os.path.join(dest_path, date, stock, "otc_lob_" + stock +  ".csv"), index=False)
            df_tick_dict[stock].to_csv(os.path.join(dest_path, date, stock, "otc_tick_" + stock + ".csv"), index=False)
    return df_lob_dict, df_tick_dict

In [20]:
df_lob_dict_1, df_tick_dict_1 = filter_otc()

In [21]:
df_lob_dict_1

{'6462':               time stock  bid1_price  ...  ask4_price  ask5_qty  ask5_price
 41830     90022839  6462       178.0  ...       180.0        20       180.5
 42825     90027608  6462       178.0  ...       180.0        20       180.5
 45930     90039859  6462       178.0  ...       180.0        20       180.5
 49334     90053522  6462       178.0  ...       180.0        21       180.5
 49664     90055118  6462       178.0  ...       180.0        21       180.5
 ...            ...   ...         ...  ...         ...       ...         ...
 1097741  132928887  6462       177.5  ...       179.5        38       180.0
 1097934  132933909  6462       177.5  ...       179.5        38       180.0
 1098298  132943952  6462       177.5  ...       179.5        38       180.0
 1098480  132948974  6462       177.0  ...       179.0        22       179.5
 1099627  133000000  6462       177.0  ...       179.0        22       179.5
 
 [1450 rows x 22 columns],
 '6488':               time stock  bid1

In [22]:
df_tick_dict_1

{'6462':              time stock  price  qty
 27351    90022839  6462  178.0    4
 36836    90302143  6462  178.5    1
 37842    90321719  6462  178.5    1
 41393    90440779  6462  178.5    1
 41395    90440798  6462  178.5    1
 ...           ...   ...    ...  ...
 236530  132928887  6462  177.5   44
 236708  132933909  6462  177.5   44
 237041  132943952  6462  177.5   44
 237209  132948974  6462  177.5   46
 238006  133000000  6462  177.5   46
 
 [339 rows x 4 columns], '6488':              time stock  price  qty
 27309    90021834  6488  403.0   12
 27725    90028843  6488  403.0    1
 27873    90031542  6488  403.5    1
 27874    90031607  6488  403.0    1
 27978    90033335  6488  403.0    2
 ...           ...   ...    ...  ...
 236828  132937726  6488  397.5  191
 237166  132947769  6488  397.5  191
 237341  132952788  6488  396.5  186
 237541  132957809  6488  396.5  188
 237990  133000000  6488  396.5  188
 
 [832 rows x 4 columns]}